In [ ]:
#ecg_wave_model.py
import os
import sys

from tensorflow.keras.layers import Lambda, Input, Dense, LSTM, RepeatVector, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import mse, binary_crossentropy, kullback_leibler_divergence
from tensorflow.keras.utils import plot_model
from keras import backend as K
from tensorflow.keras.regularizers import l1
import numpy as np
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, Lambda
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten
from tensorflow.keras.layers.convolutional import Conv1D, Conv2D, Conv3D, UpSampling2D, UpSampling3D, MaxPooling1D
from tensorflow.keras.layers.convolutional import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D

# IPython imports
from IPython.display import Image

# ml4h Imports
from ml4h.arguments import parse_args
from ml4h.tensor_generators import test_train_valid_tensor_generators
from ml4h.defines import TENSOR_EXT, IMAGE_EXT, MRI_TO_SEGMENT, MRI_SEGMENTED
from ml4h.recipes import train_multimodal_multitask, test_multimodal_multitask
from ml4h.models import make_multimodal_multitask_model, train_model_from_generators

In [ ]:
input_x = Input(shape=(5000, 12), name='in')
x = Conv1D(filters=128, kernel_size=101, activation='relu', padding='same')(input_x)
x = Conv1D(filters=128, kernel_size=101, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=51, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=51, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
conv_label = Conv1D(12, 1, activation="linear", padding='same')(x)
output_y = Activation('linear', name='out')(conv_label)
m = Model(inputs=[input_x], outputs=[output_y])
m.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m.compile(optimizer=opt, loss='logcosh')

In [ ]:
input_x = Input(shape=(5000, 12), name='in')
x = Conv1D(filters=64, kernel_size=101, activation='relu', padding='same')(input_x)
x = Conv1D(filters=64, kernel_size=101, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='same')(x)
conv_label = Conv1D(12, 1, activation="linear", padding='same')(x)
output_wave = Activation('linear', name='out')(conv_label)

alphabet_size = 60
window_size = 40

flat_x = Flatten()(x)
flat_x = Dense(16, activation='relu')(flat_x)
wave_embeds = RepeatVector(window_size)(flat_x)
#Permute((2, 1)

burn_in = Input(shape=(window_size, alphabet_size), name='burn_in')
lstm_in = concatenate([burn_in, wave_embeds])

lstm_out = LSTM(128)(lstm_in)
output_char = Dense(alphabet_size, activation='softmax')(lstm_out)

m = Model(inputs=[input_x, burn_in], outputs=[output_wave, output_char])
m.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m.compile(optimizer=opt, loss=['logcosh', 'categorical_crossentropy'])

In [ ]:
sys.argv = ['train', 
            '--tensors','/mnt/disks/ecg-text2/2019-03-30/', 
            '--input_tensors', 'ecg_rest_1lead', 
            '--output_tensors','ecg_median_1lead', 'ecg_rhythm', 'ecg_normal', 'ventricular-rate', 
            '--batch_size', '32', 
            '--epochs', '1',
            '--training_steps', '30',
            '--inspect_model',
            '--u_connect',
            '--id', 'ecg_wave_regress_and_classify']
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

In [ ]:
model = make_multimodal_multitask_model(**args.__dict__)


In [ ]:
model = train_model_from_generators(model, generate_train, generate_valid, args.training_steps,
                                        args.validation_steps, args.batch_size, args.epochs, args.patience,
                                        args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

In [ ]:
Image('./recipes_output/ecg_wave_regress_and_classify/architecture_graph_ecg_wave_regress_and_classify.png')

In [ ]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('embed').output)
intermediate_layer_model.summary()
intermediate_output = intermediate_layer_model.predict_generator(generate_test, steps=1)
print(intermediate_output.shape)

In [ ]:
embed = model.get_layer('embed').output

burn_in = Input(shape=(window_size, alphabet_size), name='burn_in')
embed_in = Input(shape=(64,), name='embed_in')
wave_embeds = RepeatVector(window_size)(embed_in)
lstm_in = concatenate([burn_in, wave_embeds])
alphabet_size = 60
window_size = 40
lstm_out = LSTM(128)(lstm_in)
output_char = Dense(alphabet_size, activation='softmax')(lstm_out)

m_caption = Model(inputs=[embed_in, burn_in], outputs=[output_char])
m_caption.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m_caption.compile(optimizer=opt, loss='categorical_crossentropy')

In [ ]:
Image('./recipes_output/ecg_wave_regress_and_classify/architecture_graph_ecg_wave_regress_and_classify.png')